<h1><b><font color = orange>Applied Data Science Capstone </font></b></h1>
<h3>Edesio Santana - Wroclaw, June 24th, 2019<h3><br>

<h3>Table of Contents</h3>
1. Executive Summary<br>
2. Introduction<br>
3. Literature Review<br>
4. Methodology<br>
5. Results<br>
6. Discussion<br>
7. Conclusion<br>
8. Acknowledgments<br>
9. References<br>
10. Appendix

<h3><b><font color = orange>1. Executive Summary</font></b></h3>
- Poland is home to Business services centers of companies from 41 countries. The city of Kraków has most of jobs in the sector with 70,000 employees, which means a 23% share in Poland. The sector created more than 56,000 jobs in Warsaw and 47,500 jobs in Wrocław. Therefore it is advantageous to <b>compare cities and recommend where other companies could setup their offices.</b><br><br>
- In this Project I am using the Foursquare API to explore <b>Wrocław</b> and <b>Kraków</b>, evaluating infrastructure and trending restaurants using a Folium to visualize the results on interactive maps.


<h3><b><font color = orange>2. Introduction</font></b></h3>
- In this project I’ve decided to leverage <b>the Foursquare location data</b> to compare differences between the cities of  Wrocław and Kraków. Our target audience are <b>people considering to move from of  Wrocław to Kraków and work within Business services centers.</b> While evaluating venues such as restaurants and obtaining additional information about other venues we expect to provide insights about which city would represent a better choice, based on Foursquare ratings.


<h3><b><font color = orange>3. Literature review</font></b></h3>
- <a href="https://absl.pl/wp-content/uploads/2019/06/raport_absl_2019_EN_190602_epub.pdf">Report “Business services sector in Poland in 2019”</a> published by the Association of Business Service Leaders I Poland. The report was done in collaboration with 200 companies with a total of 155,000 employees in their business services centers in Poland, representing 51% of all people employed in the sector, available for download.



<h3><b><font color = orange>4. Methodoogy</font></b></h3>
- The project is using data from the Foursquare API to solve the problem and obtain maps for Wrocław and Kraków. Other datasets are used in combination with the Foursquare location data to provide adequate explanation and discussion with examples provided below.

### Import necessary Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

### Define Foursquare Credentials and Version

##### Make sure that you have created a Foursquare developer account and have your credentials handy

In [2]:
CLIENT_ID = 'Y3UR25IN0Z0FBV3QJDLIAZW0WUWYKUDCQV31SQEQLTBDKU42' # your Foursquare ID
CLIENT_SECRET = 'JIZCJMER50TE5QXIQBZOSBQQGS5LQHNNCMUCAE4G2BHBYIIX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y3UR25IN0Z0FBV3QJDLIAZW0WUWYKUDCQV31SQEQLTBDKU42
CLIENT_SECRET:JIZCJMER50TE5QXIQBZOSBQQGS5LQHNNCMUCAE4G2BHBYIIX


#### We are looking at recommendations where other companies could setup their offices in two cities. We will first check near the Market square in the city of Wroclaw, obtaining latitude and longitude coordinates.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.

In [3]:
address = 'Market square Wroclaw'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.1089776 17.0326689


<a id="item1"></a>

## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

#### Now, let's assume that it is lunch time, and you are craving Italian food. So, let's define a query to search for Italian food that is within 500 metres from the Conrad Hotel. 

In [4]:
search_query = 'Sushi'
radius = 500
print(search_query + ' .... OK!')

Sushi .... OK!


#### Define the corresponding URL

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Y3UR25IN0Z0FBV3QJDLIAZW0WUWYKUDCQV31SQEQLTBDKU42&client_secret=JIZCJMER50TE5QXIQBZOSBQQGS5LQHNNCMUCAE4G2BHBYIIX&ll=51.1089776,17.0326689&v=20180604&query=Sushi&radius=500&limit=30'

#### Send the GET Request and examine the results

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d12110fe97dfb002c5dbcd6'},
 'response': {'venues': [{'id': '4d766861b483a09368629005',
    'name': '77 Sushi',
    'location': {'address': 'ul. Nożownicza 1C',
     'lat': 51.11285710355397,
     'lng': 17.031796067563022,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.11285710355397,
       'lng': 17.031796067563022}],
     'distance': 436,
     'postalCode': '50-119',
     'cc': 'PL',
     'city': 'Wrocław',
     'state': 'Województwo dolnośląskie',
     'country': 'Polska',
     'formattedAddress': ['ul. Nożownicza 1C', '50-119 Wrocław', 'Polska']},
    'categories': [{'id': '4bf58dd8d48988d1d2941735',
      'name': 'Sushi Restaurant',
      'pluralName': 'Sushi Restaurants',
      'shortName': 'Sushi',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1561465103',
    'hasPerk': False},
   {'id': '4c28ce88e19720a11b79f858',
    'n

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,4d766861b483a09368629005,ul. Nożownicza 1C,PL,Wrocław,Polska,NaN,436,"[ul. Nożownicza 1C, 50-119 Wrocław, Polska]","[{'label': 'display', 'lat': 51.11285710355397...",51.112857,17.031796,50-119,Województwo dolnośląskie,77 Sushi,v-1561465103,NaN
1,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,4c28ce88e19720a11b79f858,Odrzańska 17/1a,PL,Wrocław,Polska,NaN,438,"[Odrzańska 17/1a, Wrocław, Polska]","[{'label': 'display', 'lat': 51.11287541668924...",51.112875,17.031743,NaN,Województwo dolnośląskie,Sakana Sushi Bar,v-1561465103,NaN
2,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,51cd9d02498e7d6ae1bcd801,Rzeźnicza 34/1D,PL,Wrocław,Polska,Ruska,374,"[Rzeźnicza 34/1D (Ruska), 50-130 Wrocław, Polska]","[{'label': 'display', 'lat': 51.11033100748933...",51.110331,17.027761,50-130,Województwo dolnośląskie,Sushi Takai,v-1561465103,NaN
3,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,4f0eed61e4b04a8cd91df7e2,Odrzańska 1,PL,Wrocław,Polska,NaN,310,"[Odrzańska 1, 50-113 Wrocław, Polska]","[{'label': 'display', 'lat': 51.11152461814552...",51.111525,17.030845,50-113,Województwo dolnośląskie,Michiko Sushi Roll,v-1561465103,NaN
4,[],False,52bb3704498e894d89ba5456,"Rynek 46/47, Wroclaw, Poland",PL,NaN,Polska,NaN,192,"[Rynek 46/47, Wroclaw, Poland, Polska]","[{'label': 'display', 'lat': 51.110704, 'lng':...",51.110704,17.032870,NaN,NaN,Sushi Planet,v-1561465103,NaN


#### Define information of interest and filter dataframe

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,77 Sushi,Sushi Restaurant,ul. Nożownicza 1C,PL,Wrocław,Polska,NaN,436,"[ul. Nożownicza 1C, 50-119 Wrocław, Polska]","[{'label': 'display', 'lat': 51.11285710355397...",51.112857,17.031796,50-119,Województwo dolnośląskie,4d766861b483a09368629005
1,Sakana Sushi Bar,Sushi Restaurant,Odrzańska 17/1a,PL,Wrocław,Polska,NaN,438,"[Odrzańska 17/1a, Wrocław, Polska]","[{'label': 'display', 'lat': 51.11287541668924...",51.112875,17.031743,NaN,Województwo dolnośląskie,4c28ce88e19720a11b79f858
2,Sushi Takai,Sushi Restaurant,Rzeźnicza 34/1D,PL,Wrocław,Polska,Ruska,374,"[Rzeźnicza 34/1D (Ruska), 50-130 Wrocław, Polska]","[{'label': 'display', 'lat': 51.11033100748933...",51.110331,17.027761,50-130,Województwo dolnośląskie,51cd9d02498e7d6ae1bcd801
3,Michiko Sushi Roll,Sushi Restaurant,Odrzańska 1,PL,Wrocław,Polska,NaN,310,"[Odrzańska 1, 50-113 Wrocław, Polska]","[{'label': 'display', 'lat': 51.11152461814552...",51.111525,17.030845,50-113,Województwo dolnośląskie,4f0eed61e4b04a8cd91df7e2
4,Sushi Planet,None,"Rynek 46/47, Wroclaw, Poland",PL,NaN,Polska,NaN,192,"[Rynek 46/47, Wroclaw, Poland, Polska]","[{'label': 'display', 'lat': 51.110704, 'lng':...",51.110704,17.032870,NaN,NaN,52bb3704498e894d89ba5456
5,Sushirolka,Sushi Restaurant,Kuźnicza 25,PL,Wrocław,Polska,NaN,386,"[Kuźnicza 25, 50-138 Wrocław, Polska]","[{'label': 'display', 'lat': 51.11228757802861...",51.112288,17.034343,50-138,Województwo dolnośląskie,57127013498e87c71ff3cfdf
6,Sushi & Martini,Sushi Restaurant,NaN,PL,NaN,Polska,NaN,333,[Polska],"[{'label': 'display', 'lat': 51.111317, 'lng':...",51.111317,17.029686,NaN,NaN,529dcf28498eb70ed26a3421
7,KOBO SUSHI,Sushi Restaurant,Rynek 46/47,PL,Wrocław,Polska,NaN,170,"[Rynek 46/47, 50-116 Wrocław, Polska]","[{'label': 'display', 'lat': 51.11050948756854...",51.110509,17.032842,50-116,Województwo dolnośląskie,5509d10b498e9952c203d166
8,Sushi Lovers,Sushi Restaurant,20 Odrzańska,PL,Вроцлав,Polska,NaN,479,"[20 Odrzańska, 50-114 Вроцлав, Polska]","[{'label': 'display', 'lat': 51.113216, 'lng':...",51.113216,17.031418,50-114,Нижняя Силезия,5cf5764ca2c00b002cf650d2
9,Wasabi Sushi Shop,Grocery Store,Biskupia 4b,PL,Wroclaw,Polska,Oławska,290,"[Biskupia 4b (Oławska), 50-148 Wroclaw, Polska]","[{'label': 'display', 'lat': 51.109033, 'lng':...",51.109033,17.036819,50-148,Lower Silesia,577a3fdd498ec5cc1b3d8b5f


#### Let's visualize the Italian restaurants that are nearby

In [9]:
dataframe_filtered.name

0               77 Sushi
1       Sakana Sushi Bar
2            Sushi Takai
3     Michiko Sushi Roll
4           Sushi Planet
5             Sushirolka
6        Sushi & Martini
7             KOBO SUSHI
8           Sushi Lovers
9      Wasabi Sushi Shop
10            Hana Sushi
11          Szajnochy 11
Name: name, dtype: object

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=17) # generate map centred around the Market Square in Wroclaw

# add a red circle marker to represent the Market Square
folium.features.CircleMarker(
    [latitude, longitude],
    radius=20,
    color='red',
    popup='Market Square Wroclaw',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add japanese restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a id="item2"></a>

## 2. Explore a Given Venue
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### A. Let's explore the closest Italian restaurant -- _77 Sushi_

In [11]:
venue_id = '4d766861b483a09368629005' # ID of 77 Sushi
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4d766861b483a09368629005?client_id=Y3UR25IN0Z0FBV3QJDLIAZW0WUWYKUDCQV31SQEQLTBDKU42&client_secret=JIZCJMER50TE5QXIQBZOSBQQGS5LQHNNCMUCAE4G2BHBYIIX&v=20180604'

#### Send GET request for result

In [12]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'storeId', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4d766861b483a09368629005',
 'name': '77 Sushi',
 'contact': {'phone': '+48713417271', 'formattedPhone': '+48 71 341 72 71'},
 'location': {'address': 'ul. Nożownicza 1C',
  'lat': 51.11285710355397,
  'lng': 17.031796067563022,
  'labeledLatLngs': [{'label': 'display',
    'lat': 51.11285710355397,
    'lng': 17.031796067563022}],
  'postalCode': '50-119',
  'cc': 'PL',
  'city': 'Wrocław',
  'state': 'Województwo dolnośląskie',
  'country': 'Polska',
  'formattedAddress': ['ul. Nożownicza 1C', '50-119 Wrocław', 'Polska']},
 'canonicalUrl': 'https://foursquare.com/v/77-sushi/4d766861b483a09368629005',
 'categories': [{'id': '4bf58dd8d48988d1d2941735',
   'name': 'Sushi Restaurant',
   'pluralName': 'Sushi Restaurants',
   'shortName': 'Sushi',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d111941735',
   'name': 'Japanese Restaurant',
   'pluralName': 'Japanese Restaurants',
   'sh

### B. Get the venue's overall rating

In [13]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.2


7.2 is a good rating, but let's check the rating of the second closest Japanese restaurant.

In [14]:
venue_id = '4c28ce88e19720a11b79f858' # ID of Sakana Sushi Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.4


This restaurant has a better rating, 7.4 but let's check a third restaurant.

In [15]:
venue_id = '4c4c2c20f7cc1b8ddc1aed40' # ID of Szajnochy 11
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.0


This restaurant has a very good rating, 8.0. Let's explore this further.

### C. Get the number of tips

In [16]:
result['response']['venue']['tips']['count']

15

### D. Get the venue's tips
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

#### Create URL and send GET request. Make sure to set limit to get all tips

In [18]:
## Szajnochy 11 Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d121170c0c0840025bd26c4'},
 'response': {'tips': {'count': 15,
   'items': [{'id': '54b37c43498e9cffea330fe4',
     'createdAt': 1421048899,
     'text': 'Amazing sushi and definitely a higher standard than some other sushi places. Tasty and fresh fish. And you have the possibility to watch the sushi being made in front of you.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/54b37c43498e9cffea330fe4',
     'photo': {'id': '54b37c55498ea106d6415914',
      'createdAt': 1421048917,
      'source': {'name': 'Foursquare for Android',
       'url': 'https://foursquare.com/download/#/android'},
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'suffix': '/95753554_Ox0msdtcBLBMDm_kMTUBHk8Ql8zLne0-4KNhUkGSEK4.jpg',
      'width': 2688,
      'height': 1520,
      'visibility': 'public'},
     'photourl': 'https://fastly.4sqi.net/img/general/original/95753554_Ox0msdtcBLBMDm_kMTUBHk8Ql8zLne0-4KNhUkGSEK4.jpg',
     'lang':

#### Get tips and list of associated features

In [19]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

#### Format column width and display all tips

In [20]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Amazing sushi and definitely a higher standard than some other sushi places. Tasty and fresh fish. And you have the possibility to watch the sushi being made in front of you.,1,0,54b37c43498e9cffea330fe4,The T,a,male,95753554


Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips.

<a id="item3"></a>

## 3. Search a Foursquare User
> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### Define URL, send GET request and display features associated with user

In [21]:
user_id = '95753554' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])

In [22]:
print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])

First Name: The T
Last Name: a
Home City: Wroclaw, Poland & Moers, Germany


#### How many tips has this user submitted?

In [23]:
user_data['tips']

{'count': 302}

Wow! So it turns out that "The T" is a very active Foursquare user, with more than 302 tips.

### Get User's tips

In [24]:
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,"Well, there is no tangible border anymore. Which is great, a convenience for all. Thanks Schengen, thanks EU.",1,0,59b684a63362731c8023eeeb
1,It is a bridge. Solid experience in terms of crossing over it. Would do it again. 6/10,2,0,59b3e3772d2fd96a666f159f
2,Really good pizza. One of the best places for pizza. And I love the locale.,0,0,59b3e2fa1108ba21a3f1625e
3,It needs ticket machines. It would help with those long and slow lines. Make sure you have enough time planned in to stand in line - or you will miss your train. Otherwise nice and newly restaurated,0,0,59b3e2b3dab4b12999d79488
4,"Apparently the cashiers here are challening those from Carrefour for the title ""slowest ever"", ""die before checkout"" and ""you will never leave""",0,0,59b11d7346e1b6575548c213
5,Der Legende zufolge die kleinste Großstadt Deutschlands ;),4,0,59b11c949ef8ef4146ff4c2b
6,"Barbecue, amazing meat, full with flavours, perfect! I loved the ""Black Death"" - spicy, tasty, juicy and unique :)",0,0,5962429316ef6728f714528e
7,"Great selection of ever changing craft beers and right next to it one can find Happy Truck - the best pizza in Wrocław. Great pizza and beer, what more can you ask for ;)",2,0,595a8caa16fa040b520b568b
8,"It is okay. Had worse ones, had better ones, too. Great spot though. Wish they used some ingredients with better quality.",0,0,59577fe8c365885c8b41e048
9,The Burgers need more flavour and texture and the chili fries had the flavour and consistency of a sponge.,0,0,59555c1b6661165ff17c9c03


#### Let's get the venue for the tip with the greatest number of agree counts

In [25]:
tip_id = '54b37c43498e9cffea330fe4' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

Szajnochy 11
{'address': 'Szajnocha 11', 'lat': 51.108176718716564, 'lng': 17.028726430400464, 'labeledLatLngs': [{'label': 'display', 'lat': 51.108176718716564, 'lng': 17.028726430400464}], 'postalCode': '50-073', 'cc': 'PL', 'city': 'Wrocław', 'state': 'Województwo dolnośląskie', 'country': 'Polska', 'formattedAddress': ['Szajnocha 11', '50-073 Wrocław', 'Polska']}


### Get User's friends

In [28]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

,bio,firstName,gender,homeCity,id,lastName,lists.groups,photo.prefix,photo.suffix,superuser,tips.count
0,,Michał,male,"Wrocław, Województwo Dolnośląskie",81512541,S,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/81512541-MNBC3D0WC2RIM2XO.jpg,NaN,0
1,,Kuba,male,"Świdnica, Dolnośląskie",35649762,J,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/35649762_rw91UNTJ_C4aAkJtVtb8ki6INJMezpNn3utV3CtFs1qDxjy1bNHdcPmyc2sKtXBQQNgX_fukt.jpg,2.0,28
2,"Wroclaw maniac & travel addict: if you're coming to my area and need some help, send me a message. I may be of help to you",Krzys,male,Lower Silesian,29820439,J,"[{'type': 'created', 'count': 3, 'items': []}]",https://fastly.4sqi.net/img/user/,/29820439_4FFjGEU8_rTh_jd4T9snWNoEY0jCGWAuSBfqpvll2H-GTMkR635R4UKw9WnjTMf4BUjMUk3Bg.jpg,NaN,160
3,,Tetiana,female,"Ierápetra, ESYE43",63145808,I,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/1MUW1SYVQ4MIIOK3.jpg,NaN,3
4,,Lotterliebe,female,"Berlin, Germany",2091561,NaN,"[{'type': 'created', 'count': 17, 'items': []}]",https://fastly.4sqi.net/img/user/,/2091561_uqpgk1g6_k_v_0zsyq9C4aZELIuDNdHbdL228iGTfS6Ghrme-_lFHNxt0YbXY3kdVmkRs3wy-.jpg,NaN,112
5,,Fabio,male,"Wroclaw, Poland",80389749,P,"[{'type': 'created', 'count': 14, 'items': []}]",https://fastly.4sqi.net/img/user/,/80389749-2XH3BZBG1OUCO4XZ.jpg,NaN,7


Interesting, "The T" has friends from different countries from Europe on Foursquare. That's a real travaller!

### Retrieve the User's Profile Image

In [29]:
user_data

{'id': '95753554',
 'firstName': 'The T',
 'lastName': 'a',
 'gender': 'male',
 'canonicalUrl': 'https://foursquare.com/tomcio88',
 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
  'suffix': '/95753554-2AH2AS0PA0SNGM1V.jpg'},
 'friends': {'count': 6,
  'groups': [{'type': 'others',
    'name': 'Other friends',
    'count': 6,
    'items': [{'id': '81512541',
      'firstName': 'Michał',
      'lastName': 'S',
      'gender': 'male',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/81512541-MNBC3D0WC2RIM2XO.jpg'},
      'tips': {'count': 0},
      'lists': {'groups': [{'type': 'created', 'count': 2, 'items': []}]},
      'homeCity': 'Wrocław, Województwo Dolnośląskie',
      'bio': '',
      'contact': {}},
     {'id': '35649762',
      'firstName': 'Kuba',
      'lastName': 'J',
      'gender': 'male',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/35649762_rw91UNTJ_C4aAkJtVtb8ki6INJMezpNn3utV3CtFs1qDxjy1bNHd

In [33]:
# 1. grab prefix of photo
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://fastly.4sqi.net/img/user/300x300/95753554-2AH2AS0PA0SNGM1V.jpg')

<a id="item4"></a>

## 4. Explore a location
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**

#### Let's now find popular spots around the Japanese restaurant <b>Szajnochy 11</b>. In order to explore the area, let's start by getting the latitude and longitude values of the restaurant.

In [34]:
latitude = 51.108505 
longitude = 17.032074

#### Define URL

In [35]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=Y3UR25IN0Z0FBV3QJDLIAZW0WUWYKUDCQV31SQEQLTBDKU42&client_secret=JIZCJMER50TE5QXIQBZOSBQQGS5LQHNNCMUCAE4G2BHBYIIX&ll=51.108505,17.032074&v=20180604&radius=500&limit=30'

#### Send GET request and examine results

In [36]:
import requests

In [37]:
results = requests.get(url).json()
'There are {} around Szajnochy 11 restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Szajnochy 11 restaurant.'

#### Get relevant part of JSON

In [38]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b643fc4f964a52058a62ae3',
  'name': 'Rynek',
  'location': {'address': 'Rynek',
   'lat': 51.10969782815125,
   'lng': 17.031937186025647,
   'labeledLatLngs': [{'label': 'display',
     'lat': 51.10969782815125,
     'lng': 17.031937186025647}],
   'distance': 133,
   'postalCode': '50-065',
   'cc': 'PL',
   'city': 'Wrocław',
   'state': 'Województwo dolnośląskie',
   'country': 'Polska',
   'formattedAddress': ['Rynek', '50-065 Wrocław', 'Polska']},
  'categories': [{'id': '4bf58dd8d48988d164941735',
    'name': 'Plaza',
    'pluralName': 'Plazas',
    'shortName': 'Plaza',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4b643fc4f964a52058a62ae3-0'}

#### Process JSON and convert it to a clean dataframe

In [39]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Rynek,Plaza,Rynek,PL,Wrocław,Polska,NaN,133,"[Rynek, 50-065 Wrocław, Polska]","[{'label': 'display', 'lat': 51.10969782815125, 'lng': 17.031937186025647}]",51.109698,17.031937,50-065,Województwo dolnośląskie,4b643fc4f964a52058a62ae3
1,Tajne Komplety,Bookstore,ul. Przejście Garncarskie 2,PL,Wrocław,Polska,NaN,182,"[ul. Przejście Garncarskie 2, 50-107 Wrocław, Polska]","[{'label': 'display', 'lat': 51.11013389176684, 'lng': 17.03238714670088}]",51.110134,17.032387,50-107,Województwo dolnośląskie,4d87a00561676dcb4cb9b8e4
2,Stacja Pasibus,Burger Joint,Świdnicka 11,PL,Wrocław,Polska,NaN,82,"[Świdnicka 11, 50-066 Wrocław, Polska]","[{'label': 'display', 'lat': 51.107766, 'lng': 17.032182}]",51.107766,17.032182,50-066,Województwo dolnośląskie,569bc71c498e9972e040403b
3,Vivere Italiano,Wine Shop,Ofiar Oswiecimskich,PL,Wrocław,Polska,Świdnicka,1,"[Ofiar Oswiecimskich (Świdnicka), Wrocław, Polska]","[{'label': 'display', 'lat': 51.10850472806552, 'lng': 17.032054981020867}]",51.108505,17.032055,NaN,Województwo dolnośląskie,540c7cd7498e32e774fea065
4,Kontynuacja,Beer Bar,Ofiar Oświęcimskich 17,PL,Wrocław,Polska,NaN,28,"[Ofiar Oświęcimskich 17, 50-069 Wrocław, Polska]","[{'label': 'display', 'lat': 51.108511143559625, 'lng': 17.031659184950918}]",51.108511,17.031659,50-069,Województwo dolnośląskie,524090ffbce6a00b7f4c55b6
5,Vincent,Bakery,Oławska 8,PL,Wrocław,Polska,NaN,200,"[Oławska 8, Wrocław, Polska]","[{'label': 'display', 'lat': 51.108686, 'lng': 17.034931}]",51.108686,17.034931,NaN,Województwo dolnośląskie,54b94c02498e8e5ef2098aba
6,Mama Manoush,Gastropub,Świdnicka,PL,Wrocław,Polska,NaN,67,"[Świdnicka, Wrocław, Polska]","[{'label': 'display', 'lat': 51.10836207938017, 'lng': 17.033008933067322}]",51.108362,17.033009,NaN,Województwo dolnośląskie,56ce05c5498efad1a3157f70
7,Plac Solny,Plaza,pl. Solny,PL,Wrocław,Polska,NaN,224,"[pl. Solny, 50-060 Wrocław, Polska]","[{'label': 'display', 'lat': 51.10962820353546, 'lng': 17.029409112299632}]",51.109628,17.029409,50-060,Województwo dolnośląskie,4b64582af964a5201cac2ae3
8,Whiskey in the Jar,Steakhouse,"Rynek 23,24",PL,Wrocław,Polska,NaN,95,"[Rynek 23,24, 50-101 Wrocław, Polska]","[{'label': 'display', 'lat': 51.10936588828019, 'lng': 17.032133500891508}]",51.109366,17.032134,50-101,Województwo dolnośląskie,5374f01d498e4dcce41c82c7
9,Happy Little Truck,Pizza Place,NaN,PL,Wrocław,Polska,NaN,186,"[Wrocław, Polska]","[{'label': 'display', 'lat': 51.10737598217345, 'lng': 17.03009883628581}]",51.107376,17.030099,NaN,Województwo dolnośląskie,53627d93498e00598359dadc


#### Let's visualize these items on the map around our location

In [40]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=17) # generate map centred around Szajnochy 11


# add Szajnochy 11 as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Szajnochy 11',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

<h3><b><font color = orange>4. Results</font></b></h3>
- First we’ve got coordinates for the Market Square in Wroclaw, searching for Japanese restaurants. We’ve got the closest venues and their ratings. We’ve identified the restaurant with higher ratings and get a foursquare user tips. We’ve been able to get insights about him like how many posts he had and the  nationality of his friends in foursquare, even to look at his picture.<br>
- After that we will generate maps for the second location, compare the results and consolidate that on the report for Week 5. 

<a id="item6"></a>